In [23]:
import os
import sys
import re

# Get the current working directory
current_dir = os.getcwd()
# Add the ./src folder to the Python module search path
sys.path.append(os.path.join(current_dir, '..', 'src'))


from utils import *
from collections import Counter

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

sns.set_style('ticks')

In [14]:
input_path = '../Data/'

feature_file = 'Fingerprints/Morgan_Fingerprints_Frequency_Size50.csv'
# feature_file = 'Fingerprints/TopologicalTorsions_Fingerprints_Frequency_Size50.csv'
features_file_2 =  'leffingwell_features.npy'
CID_file = 'molecules_train_cid.npy'

# Read all copies, before and after correction; before was also downloaded from Dropbox.
mixture_file = 'Mixure_Definitions_Training_set_UPD2.csv' 
training_task_file = 'TrainingData_mixturedist.csv'

# Mordred features
features_1 = pd.read_csv(os.path.join(input_path, feature_file), index_col= 0)
features_2 = np.load(os.path.join(input_path, features_file_2))

features_CIDs = np.load(os.path.join(input_path, CID_file))

# Training dataframe
training_set = pd.read_csv(os.path.join(input_path, training_task_file))

# Mapping helper files
mixtures_IDs = pd.read_csv(os.path.join(input_path, mixture_file))

In [15]:
# Map CID to 96 dim features:
CID2features_morgan =  {CID: features_1.loc[CID].tolist() for CID in features_CIDs}
CID2features_leffingwell = {CID: features_2[i] for i, CID in enumerate(features_CIDs)}

In [16]:
X_m, y, num_mixtures, all_pairs_CIDs = format_Xy(training_set,  mixtures_IDs, CID2features_morgan, method = 'sum')
X_l, _, _, _ = format_Xy(training_set,  mixtures_IDs, CID2features_leffingwell, method = 'sum')

In [17]:
# Convert the input pairs to a suitable format for training
X_pairs_m = np.array([(np.concatenate((x1, x2))) for x1, x2 in X_m])
X_pairs_l = np.array([(np.concatenate((x1, x2))) for x1, x2 in X_l])


In [20]:
X_features = X_pairs_l
feature_dim = X_features.shape[1]

y_true = np.array(y)

# Feature names
feature_names = [f'Lw_dim{i}_1' for i in range(113)] +  [f'Lw_dim{i}_2' for i in range(113)]
name2index = dict(zip(feature_names, list(range(len(feature_names)))))

In [21]:
# Start the loop:
n_folds = 10
seed = 314159
num_removal = 300
iter = 0

while num_removal != 0:
    print(f'Starting {iter + 1}th round reduction! \n')
    print(f'Number of features used now: {feature_dim}')

    rf_pred_list = []
    y_true_list = []
    low_importance_features = []
    kf_importances = []

    # Perform k-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

    for train_index, test_index in kf.split(X_features):
        X_train, X_test = X_features[train_index], X_features[test_index]
        y_train, y_test = y_true[train_index], y_true[test_index]
        
        # Train the Random Forest regressor
        rf = RandomForestRegressor(n_estimators=100,
                                    random_state=seed)
        rf.fit(X_train, y_train)
        
        rf_pred = rf.predict(X_test)
        rf_pred_list.extend(rf_pred)
        y_true_list.extend(y_test)
 
        # Get the feature importances
        importances = rf.feature_importances_
        kf_importances.append(importances)
        # Find the indices of features with importance < 0.00010
        low_importance_indices_fold = np.where(importances < 0.00100)[0]

        # Get the names of the low importance features
        low_importance_features_fold = [feature_names[i] for i in low_importance_indices_fold]
        # Append the low importance features for this fold to the list
        low_importance_features.append(low_importance_features_fold)

    # Calculate the correlation and R^2 for Random Forest
    rf_corr = np.corrcoef(rf_pred_list, y_true_list)[0, 1]
    rf_rmse = np.sqrt(mean_squared_error(np.array(y_true_list), np.array(rf_pred_list)))

    print(f"Random Forest - R: {rf_corr:.3f}")
    print(f"Random Forest - RMSE: {rf_rmse:.3f}")
    print()

    # Find the features that appear in all the low importance lists
    consistent_low_importance_features = set.intersection(*map(set, low_importance_features))

    # If the Descriptor shows up for both molecules, we select it:
    feature_candidates = [c[:-2] for c in consistent_low_importance_features]
    element_counts = Counter(feature_candidates)
    descriptors_to_remove = [element for element, count in element_counts.items() if count == 2]

    features_to_remove_1 = [f'{d}_1' for d in descriptors_to_remove]
    features_to_remove_2 = [f'{d}_2' for d in descriptors_to_remove]

    features_to_remove = features_to_remove_1 + features_to_remove_2
    idx_to_remove = [name2index[f] for f in features_to_remove]
    num_removal = len(features_to_remove)

    print(f'Remove {num_removal} of features;')

    feature_dim = feature_dim - len(features_to_remove)
    print(f'Left with {feature_dim} features.')
    print()

    # Update feature matrix:
    X_features = np.delete(X_features, idx_to_remove, axis=1)
    feature_names = [feature for feature in feature_names if feature not in features_to_remove]
    name2index = dict(zip(feature_names, list(range(len(feature_names)))))

    # iter += 1
    # np.save(f'Output/feature_reduction/feature_names_iter_{iter}.npy', feature_names)


Starting 1th round reduction! 

Number of features used now: 226
Random Forest - R: 0.442
Random Forest - RMSE: 0.140

Remove 34 of features;
Left with 192 features.

Starting 1th round reduction! 

Number of features used now: 192
Random Forest - R: 0.447
Random Forest - RMSE: 0.140

Remove 0 of features;
Left with 192 features.



In [27]:
extracted_numbers = [int(re.search(r'dim(\d+)_', name).group(1)) for name in feature_names]


In [31]:
to_keep = list(set(extracted_numbers))

In [34]:
features_2.shape

(162, 113)

In [35]:
features_2_kept = features_2[:, to_keep]

In [36]:
features_2_kept.shape

(162, 96)